# DataFrames Basics

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [ ]:
ls -l # check the .tgz is there

total 1675960
drwxr-xr-x  2 root      root      4096 Dec  7 16:02 dataset/
-rwxr-xr-x  1 root      root  30053267 May  4  2021 ngrok*
-rw-r--r--  1 root      root  13832437 Dec  7 15:53 ngrok-stable-linux-amd64.zip
-rw-r--r--  1 root      root  13832437 Dec  7 15:54 ngrok-stable-linux-amd64.zip.1
-rw-r--r--  1 root      root  13832437 Dec  7 16:22 ngrok-stable-linux-amd64.zip.2
drwxr-xr-x  1 root      root      4096 Dec  5 14:37 sample_data/
drwxr-xr-x 13 110302528 1000      4096 Oct 15 09:41 spark-3.3.1-bin-hadoop2/
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz.1
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz.2
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz.3
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz.4
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-h

In [ ]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [ ]:
!pip install -q findspark

In [ ]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/cars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2022/master/datasets/movies.json -P /dataset

In [ ]:
ls -l /dataset

total 76
-rw-r--r-- 1 root root 74910 Dec  7 17:15 cars.json


Read JSON file

In [ ]:
carsDF = spark.read.option("inferSchema", True).json("/dataset/cars.json") # inferSchema requires one extra pass over the data

# if None is set, it uses de default value (default = False) you can also pass the schema manually

## Examples

Showing a DF

In [ ]:
carsDF.show(2)
carsDF.printSchema()

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 2 rows

root
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: long (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- Miles_per_Gallon: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- 

Get Rows

In [ ]:
carsDF.take(2)

[Row(Acceleration=12.0, Cylinders=8, Displacement=307.0, Horsepower=130, Miles_per_Gallon=18.0, Name='chevrolet chevelle malibu', Origin='USA', Weight_in_lbs=3504, Year='1970-01-01'),
 Row(Acceleration=11.5, Cylinders=8, Displacement=350.0, Horsepower=165, Miles_per_Gallon=15.0, Name='buick skylark 320', Origin='USA', Weight_in_lbs=3693, Year='1970-01-01')]

Count

In [ ]:
carsDF.count()

406

Schema

In [ ]:
# obtain a schema
carsSchema = carsDF.schema
print(type(carsSchema))
print(carsSchema)

<class 'pyspark.sql.types.StructType'>
StructType([StructField('Acceleration', DoubleType(), True), StructField('Cylinders', LongType(), True), StructField('Displacement', DoubleType(), True), StructField('Horsepower', LongType(), True), StructField('Miles_per_Gallon', DoubleType(), True), StructField('Name', StringType(), True), StructField('Origin', StringType(), True), StructField('Weight_in_lbs', LongType(), True), StructField('Year', StringType(), True)])


Custom Schemas

In [ ]:
example = spark.sparkContext.parallelize([("chevrolet chevelle malibu",18,"1970-01-01","USA"),
    ("buick skylark 320",15,"1970-01-01","USA")])

In [ ]:
exampleDF = spark.createDataFrame(example)
exampleDF.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)



With columns names

In [ ]:
names = list(["name", "weight", "date", "country"])

In [ ]:
example2DF = example.toDF(names)
example2DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: long (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
# importing sql types
from pyspark.sql.types import *

In [ ]:
# custom schema
customSchema = StructType([ \
    StructField('name', StringType(), True), \
    StructField('weight', StringType(), True), \
    StructField('date', StringType(), True), \
    StructField('country', StringType(), True)])

In [ ]:
example3DF = spark.createDataFrame(example, customSchema)
example3DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
example3DF.show(2, False)

+-------------------------+------+----------+-------+
|name                     |weight|date      |country|
+-------------------------+------+----------+-------+
|chevrolet chevelle malibu|18    |1970-01-01|USA    |
|buick skylark 320        |15    |1970-01-01|USA    |
+-------------------------+------+----------+-------+



In [ ]:
# we can also specify schema with DDL (Data Definition Language)
customSchema2 = "`name` STRING NOT NULL, `weight` INT, `date` STRING, `country` STRING"

In [ ]:
example4DF = spark.createDataFrame(example, customSchema2)
example4DF.printSchema()

root
 |-- name: string (nullable = false)
 |-- weight: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
print(type(example2DF.collect()[0]["weight"]))
print(type(example3DF.collect()[0]["weight"]))

<class 'int'>
<class 'str'>


## Exercises
1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

Exercise 1

In [ ]:
smartphones = spark.sparkContext.parallelize([
    ("Samsung", "Galaxy S10", "Android", 12),
    ("Apple", "iPhone X", "iOS", 13),
    ("Nokia", "3310", "THE BEST", 0)])

In [ ]:
smartphonesDF = smartphones.toDF(["Make", "Model", "Platform", "CameraMegapixels"])

In [ ]:
smartphonesDF.show()

+-------+----------+--------+----------------+
|   Make|     Model|Platform|CameraMegapixels|
+-------+----------+--------+----------------+
|Samsung|Galaxy S10| Android|              12|
|  Apple|  iPhone X|     iOS|              13|
|  Nokia|      3310|THE BEST|               0|
+-------+----------+--------+----------------+



Exercise 2

In [ ]:
moviesDF = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load("/dataset/movies.json")

In [ ]:
moviesDF.printSchema()
print(f"The Movies DF has {moviesDF.count()} rows")

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)

The Movies DF has 3201 rows
